In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# === INPUT ===
C = 2  # Concentration, g/L
MW = 8  # Molecular Weight, MDa
eta7_Exp = 15.653  # Experimental η@7.3, cP
eta7_Exp_D = 7.354  # Degraded η@7.3, cP

# === CONSTANTS ===
alpha = 0.763
Temp = 298  # Temperature in Kelvin
eta_in = np.exp(-3.7188 + (578.919 / (-137.546 + Temp)) + (0.0608 * (0 + 0)**1.3533))

# === CALCULATIONS ===
c_med = C * 0.5
ceta = np.arange(0.1, 100.1, 0.1)
eta = ceta / c_med

eta_0 = 1 + ceta + 0.582 * ceta**2.009 + 0.022 * ceta**4
etab = eta / (2**alpha)
cetab = etab * c_med
eta_0_FD = 1 + cetab + 0.582 * cetab**2.009 + 0.022 * cetab**4

beta = np.log(eta_0_FD) - np.log(eta_0)

parc = -0.08212 * ceta
parcb = -0.08212 * cetab
n = 1 - (0.6187 - 0.5203 * np.exp(parc))
n_FD = 1 - (0.6187 - 0.5203 * np.exp(parcb))

gama = (n / n_FD) - 1

parc2 = (n - 1) / 2
parc2b = (n_FD - 1) / 2

l_d = 0.251 + 1.54 * MW * ceta / (C * Temp)
l_db = 0.251 + 1.54 * MW * cetab / (c_med * Temp)

l = l_d * (0.810 + 0.0230 * ceta**2.438)
lb = l_db * (0.810 + 0.0230 * cetab**2.438)

omega = np.log(lb) - np.log(l)

parc1 = l * 7.3
parc1b = lb * 7.3

eta7 = 1 + (eta_0 - 1) * (1 + parc1)**parc2
eta7b = 1 + (eta_0_FD - 1) * (1 + parc1b)**parc2b

rho = np.log(eta7b) - np.log(eta7)

# === INTERPOLATIONS BASED ON η@7.3 ===
ceta_exp = np.interp(eta7_Exp, eta7, ceta)
beta_find = np.interp(ceta_exp, ceta, beta)
gama_find = np.interp(ceta_exp, ceta, gama)
omega_find = np.interp(ceta_exp, ceta, omega)
rho_find = np.interp(ceta_exp, ceta, rho)

# === Carreau-Yasuda Parameters for Given η@7.3 ===
eta_0C = eta_in + ceta_exp + 0.582 * ceta_exp**2.009 + 0.022 * ceta_exp**4
parcC = -0.08212 * ceta_exp
nC = 1 - (0.6187 - 0.5203 * np.exp(parcC))
parc2C = (nC - 1) / 2
l_dC = 0.251 + 1.54 * MW * ceta_exp / (C * Temp)
lC = l_dC * (0.810 + 0.0230 * ceta_exp**2.438)

# === Shear Rate Range ===
shear = np.concatenate((np.arange(0.01, 1, 0.01), np.arange(1, 10001, 1)))

# === Undegraded Polymer Curve ===
etaC = eta_in + (eta_0C - eta_in) * (1 + (lC * shear)**2)**((nC - 1) / 2)

# === Degraded Polymer Curve ===
FD = np.log(eta7_Exp_D / eta7_Exp) / rho_find
eta_0_D = np.exp(beta_find * FD) * eta_0C
n_D = np.exp(gama_find * FD) * nC
la_D = np.exp(omega_find * FD) * lC
eta_D = eta_in + (eta_0_D - eta_in) * (1 + (la_D * shear)**2)**((n_D - 1) / 2)

# === Final Output Table ===
carreau_df = pd.DataFrame({
    'shear': shear,
    'etaC': etaC,
    'eta_D': eta_D,
    'FD': FD
})

# === Print Head ===
print("\nFirst rows of Carreau-Yasuda Data:")
print(carreau_df.head())

# === Save to TXT ===
carreau_df.to_csv("carreau_data.txt", sep="\t", index=False)
print("\nCarreau-Yasuda data saved to 'carreau_data.txt'")

# === Plotly Plot ===
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=shear, y=etaC, mode='lines', name='Polymer UD, cP',
    line=dict(shape='spline', width=3, color='blue')
))

fig.add_trace(go.Scatter(
    x=shear, y=eta_D, mode='lines', name='Polymer D, cP',
    line=dict(shape='spline', width=3, color='red', dash='dash')
))

fig.update_layout(
    title='Carreau-Yasuda Plot (PAMA Method)',
    xaxis_title='Shear Rate (s⁻¹)',
    yaxis_title='Viscosity η (cP)',
    xaxis_type='log',
    yaxis_type='log',
    template='plotly_white',
    legend=dict(x=0.05, y=0.95)
)

fig.show()



First rows of Carreau-Yasuda Data:
   shear       etaC     eta_D        FD
0   0.01  18.415464  8.168587  0.822919
1   0.02  18.415323  8.168548  0.822919
2   0.03  18.415088  8.168482  0.822919
3   0.04  18.414760  8.168391  0.822919
4   0.05  18.414337  8.168273  0.822919

Carreau-Yasuda data saved to 'carreau_data.txt'
